# Chapter 1 - Representing Graphs

## Graph Structure

## The Adjacency List Representation

In [ ]:
g: list = [[1, 3, 4], [0, 2, 4], [1, 4], [0, 4], [0, 1, 2, 3]]
g

[[1, 3, 4], [0, 2, 4], [1, 4], [0, 4], [0, 1, 2, 3]]

In [ ]:
class Edge:
    def __init__(self, from_node: int, to_node: int, weight: float):
        self.from_node: int = from_node
        self.to_node: int = to_node
        self.weight: float = weight

---

Exploratory:

In [ ]:
d = {'a': 1, 'b': 2}
d.values()

dict_values([1, 2])

In [ ]:
list(d.values())

[1, 2]

In [ ]:
# Alt:
(list)(d.values())

[1, 2]

In [ ]:
(d.values())

dict_values([1, 2])

In [ ]:
d.keys()

dict_keys(['a', 'b'])

In [ ]:
list(d.keys())

['a', 'b']

In [ ]:
# Alt:
(list)(d.keys())

['a', 'b']

In [ ]:
(d.keys())

dict_keys(['a', 'b'])

---

Exploratory:

In [ ]:
l = [3, 5, 4]
# Not in-place:
sorted(l)

[3, 4, 5]

In [ ]:
l

[3, 5, 4]

In [ ]:
# In-place:
l.sort()

In [ ]:
l

[3, 4, 5]

---

In [ ]:
# Exploratory:
print(d)
print(len(d))

{'a': 1, 'b': 2}
2


In [ ]:
from typing import Union

class Node:
    def __init__(self, index: int, label: Union[int, str, object]=None):
        self.index: int = index
        self.edges: dict = {}
        self.label: Union[int, str, object] = label

    def num_edges(self) -> int:
        return len(self.edges)

    def get_edge(self, neighbor: int) -> Union[Edge, None]:
        if neighbor in self.edges:
            return self.edges[neighbor]
        return None

    def add_edge(self, neighbor: int, weight: float):
        self.edges[neighbor] = Edge(self.index, neighbor, weight)

    def remove_edge(self, neighbor: int):
        if neighbor in self.edges:
            del self.edges[neighbor]

    def get_edge_list(self) -> list:
        return list(self.edges.values())

    def get_sorted_edge_list(self) -> list:
        result: list = []
        neighbors: list = list(self.edges.keys())
        neighbors.sort()
        for n in neighbors:
            result.append(self.edges[n])
        return result

**Note:** In the above class, `neighbor` refers to (the index of) a neighboring node.

In [ ]:
class Graph:
    def __init__(self, num_nodes: int, undirected: bool=False):
        self.num_nodes: int = num_nodes
        self.undirected: bool = undirected
        self.nodes: list = [Node(j) for j in range(num_nodes)]
        # Note: The index of a particular item in the above list will match the index attribute of the corresponding `Node` object.

    def get_edge(self, from_node: int, to_node: int) -> Union[Edge, None]:
        if from_node < 0 or from_node >= self.num_nodes:
            raise IndexError
        if to_node < 0 or to_node >= self.num_nodes:
            raise IndexError
        return self.nodes[from_node].get_edge(to_node) # This will return `None` if an edge from `from_node` to `to_node` doesn't exist.

    def is_edge(self, from_node: int, to_node: int) -> bool:
        return self.get_edge(from_node, to_node) is not None

    def make_edge_list(self) -> list:
        all_edges: list = []
        for node in self.nodes:
            all_edges.extend(node.get_edge_list())
        return all_edges

    def insert_edge(self, from_node: int, to_node: int, weight: float):
        if from_node < 0 or from_node >= self.num_nodes:
            raise IndexError
        if to_node < 0 or to_node >= self.num_nodes:
            raise IndexError
        self.nodes[from_node].add_edge(to_node, weight)
        if self.undirected:
            self.nodes[to_node].add_edge(from_node, weight)

    def remove_edge(self, from_node: int, to_node: int):
        if from_node < 0 or from_node >= self.num_nodes:
            raise IndexError
        if to_node < 0 or to_node >= self.num_nodes:
            raise IndexError
        self.nodes[from_node].remove_edge(to_node)
        if self.undirected:
            self.nodes[to_node].remove_edge(from_node)

    def insert_node(self, label:Union[int, str, object]=None) -> Node:
        new_node: Node = Node(self.num_nodes, label=label)
        self.nodes.append(new_node)
        self.num_nodes += 1
        return new_node

    def make_copy(self) -> Graph:
        g2: Graph = Graph(self.num_nodes, self.undirected) # Note: We're instantiating a `Graph` object from within the `Graph` class.
        for node in self.nodes:
            g2.nodes[node.index].label = node.label
            for edge in node.get_edge_list():
                g2.insert_edge(edge.from_node, edge.to_node, edge.weight)
        return g2

**Note:**

1. There is no `remove_node` method.
2. The `insert_edge` method can be used to (i) add a new edge and (ii) update an existing edge weight (by replacing the existing `Edge` object with a new one).

In [ ]:
g: Graph = Graph(5, undirected=False)
g.insert_edge(0, 1, 1.0)
g.insert_edge(0, 3, 1.0)
g.insert_edge(0, 4, 3.0)
g.insert_edge(1, 2, 2.0)
g.insert_edge(1, 4, 1.0)
g.insert_edge(3, 4, 3.0)
g.insert_edge(4, 2, 3.0)
g.insert_edge(4, 3, 3.0)

## The Adjacency Matrix Representation

In [ ]:
class GraphMatrix:
    def __init__(self, num_nodes: int, undirected: bool=False):
        self.num_nodes: int = num_nodes
        self.undirected: bool = undirected
        self.connections: list = [[0.0] * num_nodes for _ in range(num_nodes)]

    def get_edge(self, from_node: int, to_node: int) -> float:
        if from_node < 0 or from_node >= self.num_nodes:
            raise IndexError
        if to_node < 0 or to_node >= self.num_nodes:
            raise IndexError
        return self.connections[from_node][to_node]

    def set_edge(self, from_node: int, to_node: int, weight: float):
        if from_node < 0 or from_node >= self.num_nodes:
            raise IndexError
        if to_node < 0 or to_node >= self.num_nodes:
            raise IndexError
        self.connections[from_node][to_node] = weight
        if self.undirected:
            self.connections[to_node][from_node] = weight

**Note:** If the above class, there is no provision to add a node after the graph has been instantiated.

In [ ]:
g: GraphMatrix = GraphMatrix(5, undirected=False)
g.set_edge(0, 1, 1.0)
g.set_edge(0, 3, 1.0)
g.set_edge(0, 4, 3.0)
g.set_edge(1, 2, 2.0)
g.set_edge(1, 4, 1.0)
g.set_edge(3, 4, 3.0)
g.set_edge(4, 2, 3.0)
g.set_edge(4, 3, 3.0)

## Why This Matters